Effective Area:
Simply extracted per energy bin and added to the output without transformation.

Energy Migration Parameters (manipulated/approximated):

    The MATRIX field in the energy dispersion HDU gives you a 2D probability distribution over true energy and "migra" (reconstructed/true energy).

    For each energy bin, it calculates:

        Mean (emig_mu_loc) = ∑ (migra * weight)

        Standard deviation (emig_mu_scale)

    This approximates the full migration PDF as a Gaussian-like summary.

Model Labeling:

    It assigns "approx" in the emig_model column to signal this is not an exact functional fit like Moyal, but just a statistical approximation.

In [34]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from scipy.stats import moyal, skewnorm
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

base_path = "C:/Users/a.nakhle/sst1mpipe/sst1mpipe/source_simulation/fits_files"
files_and_zd = [
    ("SST1M_stereo_Zen20deg_gcutenergydep_irfs.fits", 20.0),
    ("SST1M_stereo_Zen30deg_gcutenergydep_irfs.fits", 30.0),
    ("SST1M_stereo_Zen40deg_gcutenergydep_irfs.fits", 40.0),
    ("SST1M_stereo_Zen50deg_gcutenergydep_irfs.fits", 50.0),
    ("SST1M_stereo_Zen60deg_gcutenergydep_irfs.fits", 60.0),
]

results = []

for filename, zd_deg in files_and_zd:
    path = os.path.join(base_path, filename)
    print(f"Processing {filename} (ZD = {zd_deg})")
    with fits.open(path) as hdul:
        edisp_hdu = hdul['ENERGY DISPERSION']
        row = edisp_hdu.data[0]  # single row

        energ_lo = np.array(row['ENERG_LO']).flatten()
        energ_hi = np.array(row['ENERG_HI']).flatten()
        migra_lo = np.array(row['MIGRA_LO']).flatten()
        migra_hi = np.array(row['MIGRA_HI']).flatten()
        theta_lo = np.array(row['THETA_LO']).flatten()
        theta_hi = np.array(row['THETA_HI']).flatten()
        matrix = np.array(row['MATRIX'])  # shape (n_theta, n_energy, n_migra)

        migra_centers = 0.5 * (migra_lo + migra_hi)
        n_theta, n_energy, _ = matrix.shape

        for i_theta in range(n_theta):
            for i_e in range(n_energy):
                elo = energ_lo[i_e]
                ehi = energ_hi[i_e]
                hist = matrix[i_theta, i_e, :]

                if np.sum(hist) == 0 or not np.all(np.isfinite(hist)):
                    continue

                x = np.log10(migra_centers)
                y = hist / np.sum(hist)

                mask = np.isfinite(x) & np.isfinite(y)
                x = x[mask]
                y = y[mask]

                if len(x) < 4:
                    continue

                best_model = None
                best_err = np.inf
                best_params = None

                # Try Moyal with constraints
                try:
                    popt = curve_fit(
                        lambda x, loc, scale: moyal.pdf(x, loc, scale),
                        x, y, p0=[0, 0.5],
                        bounds=([-2, 0.05], [2, 2])
                    )[0]
                    y_pred = moyal.pdf(x, *popt)
                    err = mean_squared_error(y, y_pred)
                    if err < best_err:
                        best_model = "moyal"
                        best_err = err
                        best_params = (popt[0], popt[1], np.nan)
                except:
                    pass

                # Try Skewnorm with constraints
                try:
                    popt = curve_fit(
                        lambda x, a, loc, scale: skewnorm.pdf(x, a, loc, scale),
                        x, y, p0=[0, 0, 0.5],
                        bounds=([-10, -2, 0.05], [10, 2, 2])
                    )[0]
                    y_pred = skewnorm.pdf(x, *popt)
                    err = mean_squared_error(y, y_pred)
                    if err < best_err:
                        best_model = "skewnorm"
                        best_err = err
                        best_params = (popt[1], popt[2], popt[0])  # loc, scale, a
                except:
                    pass

                if best_model is not None:
                    results.append({
                        "ZD_deg": zd_deg,
                        "Theta_min_deg": theta_lo[i_theta],
                        "Theta_max_deg": theta_hi[i_theta],
                        "Etrue_min_TeV": elo,
                        "Etrue_max_TeV": ehi,
                        "Aeff_m2": np.nan,
                        "emig_mu_loc": best_params[0],
                        "emig_mu_scale": best_params[1],
                        "emig_mu_a": best_params[2],
                        "emig_model": best_model,
                        "fit_mse": best_err
                    })

df = pd.DataFrame(results)
csv_output = "sst1m_energy_migration_fits_summary.csv"
df.to_csv(csv_output, index=False)
print(f"Extraction completed. Saved to '{csv_output}' with {len(df)} entries.")


Processing SST1M_stereo_Zen20deg_gcutenergydep_irfs.fits (ZD = 20.0)
Processing SST1M_stereo_Zen30deg_gcutenergydep_irfs.fits (ZD = 30.0)
Processing SST1M_stereo_Zen40deg_gcutenergydep_irfs.fits (ZD = 40.0)
Processing SST1M_stereo_Zen50deg_gcutenergydep_irfs.fits (ZD = 50.0)
Processing SST1M_stereo_Zen60deg_gcutenergydep_irfs.fits (ZD = 60.0)
Extraction completed. Saved to 'sst1m_energy_migration_fits_summary.csv' with 1305 entries.


In [35]:
df

,ZD_deg,Theta_min_deg,Theta_max_deg,Etrue_min_TeV,Etrue_max_TeV,Aeff_m2,emig_mu_loc,emig_mu_scale,emig_mu_a,emig_model,fit_mse
0,20.0,0.00,0.75,0.328968,0.388325,NaN,-1.789815,1.395252,-5.742743,skewnorm,0.011702
1,20.0,0.00,0.75,0.388325,0.458391,NaN,-1.793713,1.337491,-5.817160,skewnorm,0.007189
2,20.0,0.00,0.75,0.458391,0.541100,NaN,-1.794327,1.338323,-5.806233,skewnorm,0.006413
3,20.0,0.00,0.75,0.541100,0.638732,NaN,-1.794168,1.338069,-5.808624,skewnorm,0.005643
4,20.0,0.00,0.75,0.638732,0.753980,NaN,-1.805863,1.335701,-5.641219,skewnorm,0.002199
...,...,...,...,...,...,...,...,...,...,...,...
1300,60.0,5.25,6.00,31.475859,35.482874,NaN,-2.000000,2.000000,-1.271406,skewnorm,0.018094
1301,60.0,5.25,6.00,35.482874,40.000000,NaN,-1.957098,1.512086,-5.283836,skewnorm,0.010065
1302,60.0,5.25,6.00,40.000000,45.092176,NaN,-2.000000,2.000000,-1.497326,skewnorm,0.011137
1303,60.0,5.25,6.00,50.832608,57.303822,NaN,-2.000000,2.000000,-1.553954,skewnorm,0.019972


In [40]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from scipy.stats import moyal, skewnorm
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

base_path = "C:/Users/a.nakhle/sst1mpipe/sst1mpipe/source_simulation/fits_files"
files_and_zd = [
    ("SST1M_stereo_Zen20deg_gcutenergydep_irfs.fits", 20.0),
    ("SST1M_stereo_Zen30deg_gcutenergydep_irfs.fits", 30.0),
    ("SST1M_stereo_Zen40deg_gcutenergydep_irfs.fits", 40.0),
    ("SST1M_stereo_Zen50deg_gcutenergydep_irfs.fits", 50.0),
    ("SST1M_stereo_Zen60deg_gcutenergydep_irfs.fits", 60.0),
]

results = []

for filename, zd_deg in files_and_zd:
    path = os.path.join(base_path, filename)
    with fits.open(path) as hdul:
        edisp_data = hdul["ENERGY DISPERSION"].data[0]
        energ_lo = np.array(edisp_data["ENERG_LO"]).flatten()
        energ_hi = np.array(edisp_data["ENERG_HI"]).flatten()
        migra_lo = np.array(edisp_data["MIGRA_LO"]).flatten()
        migra_hi = np.array(edisp_data["MIGRA_HI"]).flatten()
        matrix = np.array(edisp_data["MATRIX"])  # shape: (n_theta, n_energy, n_migra)

        migra_centers = 0.5 * (migra_lo + migra_hi)
        n_theta, n_energy, _ = matrix.shape

        for i_e in range(n_energy):
            elo = energ_lo[i_e]
            ehi = energ_hi[i_e]
            hist = matrix[0, i_e, :]  # assume theta index 0

            if np.sum(hist) == 0 or not np.all(np.isfinite(hist)):
                continue

            x = np.log10(migra_centers)
            y = hist / np.sum(hist)

            mask = np.isfinite(x) & np.isfinite(y)
            x = x[mask]
            y = y[mask]

            if len(x) < 4:
                continue

            best_model = None
            best_err = np.inf
            best_params = None

            # Fit Moyal
            try:
                popt = curve_fit(
                    lambda x, loc, scale: moyal.pdf(x, loc, scale),
                    x, y,
                    p0=[0, 1],
                    bounds=([-2, 0.01], [2, 5])
                )[0]
                y_pred = moyal.pdf(x, *popt)
                err = mean_squared_error(y, y_pred)
                best_model = "moyal"
                best_err = err
                best_params = (popt[0], popt[1], np.nan)
            except:
                pass

            # Try Skewnorm if Moyal is bad
            try:
                popt = curve_fit(
                    lambda x, loc, scale, a: skewnorm.pdf(x, a, loc, scale),
                    x, y,
                    p0=[0, 1, 0],
                    bounds=([-2, 0.01, -10], [2, 5, 10])
                )[0]
                y_pred = skewnorm.pdf(x, *popt)
                err = mean_squared_error(y, y_pred)
                if err < best_err:
                    best_model = "skewnorm"
                    best_err = err
                    best_params = (popt[1], popt[2], popt[0])  # loc, scale, a
            except:
                continue

            if best_model is not None:
                results.append({
                    "ZD_deg": zd_deg,
                    "Etrue_min_TeV": elo,
                    "Etrue_max_TeV": ehi,
                    "Aeff_m2": np.nan,
                    "emig_mu_loc": best_params[0],
                    "emig_mu_scale": best_params[1],
                    "emig_mu_a": best_params[2],
                    "emig_model": best_model
                })

# Save to CSV
df = pd.DataFrame(results)
df.to_csv("sst1m_energy_migration_fits_summary.csv", index=False)
print(f"Saved {len(df)} rows to 'sst1m_energy_migration_fits_summary.csv'.")


Saved 0 rows to 'sst1m_energy_migration_fits_summary.csv'.
